In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools as it

import re
import os

In [ ]:
FILE = 'wireless'
TOTAL_TIME = 100

In [ ]:
def get_logs(file_name=FILE+'.tr'):
    with open(file_name, 'r') as trace:
        logs = [re.split(' |\t', line.strip()) for line in trace]
    return logs

In [ ]:
def make_node(node):
    return '_' + str(ord(node) - ord('A')) + '_'

def is_destination_recieve(log):
    return log[0] == 'r' and (log[2] == make_node('H') or log[2] == make_node('I')) and log[3] == 'AGT' and log[7] == 'cbr'

def is_source_send(log):
    return log[0] == 's' and (log[2] == make_node('A') or log[2] == make_node('D')) and log[3] == 'AGT' and log[7] == 'cbr'

In [ ]:
def calculate_throughput(logs, packet_size):
    recieves = 0
    for log in logs:
        if is_destination_recieve(log):
            recieves += 1
    return packet_size * recieves / TOTAL_TIME

In [ ]:
def calculate_PTR(logs):
    recieves = 0
    sends = 0
    for log in logs:
        if is_destination_recieve(log):
            recieves += 1
        elif is_source_send(log):
            sends += 1
    return recieves / sends

In [ ]:
def calculate_delay(logs):
    delay = 0
    recieves = 0
    start_time = {}
    for log in logs:
        if is_destination_recieve(log):
            delay += float(log[1]) - start_time[log[6]]
            recieves += 1
        elif is_source_send(log):
            start_time[log[6]] = float(log[1])
    return delay / recieves

In [ ]:
def prepare_cmd(initial_cmd, bandwidth, error_rate, packet_size):
    initial_cmd += str(bandwidth) + 'Mb '
    initial_cmd += str(error_rate) + ' '
    initial_cmd += str(packet_size) + 'Kb'
    return initial_cmd

In [ ]:
def delete_outputs(file=FILE):
    if os.path.exists(file+'.nam'):
        os.remove(file+'.nam')
    if os.path.exists(file+'.tr'):
        os.remove(file+'.tr')

In [ ]:
results = pd.DataFrame(columns = ['Bandwidth(Mb/s)', 'Error Rate', 'Packet Size(Kb)', 'Throughput', 'PTR', 'Average End-To-End Delay'])
results = results.set_index(['Bandwidth(Mb/s)', 'Error Rate', 'Packet Size(Kb)'])

In [ ]:
def run(tcl=FILE+'.tcl'):
    global results
    bandwidths = np.array([1.5])#, 15, 55])
    error_rates = np.linspace(0.001, 0.01, 10)
    packet_sizes = np.array([64, 256, 512])
    initial_cmd = 'ns ' + tcl + ' '
    for bandwidth, error_rate, packet_size in it.product(bandwidths, error_rates, packet_sizes):
        cmd = prepare_cmd(initial_cmd, bandwidth, error_rate, packet_size)
        print(cmd)
        print("----------------------------------------------")
        cmd += ' >/dev/null 2>&1'
        os.system(cmd)
        
        logs = get_logs()
        throughput = calculate_throughput(logs, packet_size)
        PTR = calculate_PTR(logs)
        delay = calculate_delay(logs)
        results.loc[bandwidth, error_rate, packet_size] = [throughput, PTR, delay]
        
        delete_outputs()

In [ ]:
run()

In [ ]:
results